# **Otimização - Toy Experiment: Mitigação de Crosstalk**
## **Notebook de Geração de Dados**

**Disciplina:** PPGEE0016 - Otimização

**Alunos:** André Paiva, Josias Souza, Victor Emanuel Paz

In [111]:
import sys
sys.path.insert(1, "../crosstalk/")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from functions import *
from XTconstants import *

In [120]:
def signals_gen(signals_amount=100, n_samples=4, sampling_period=25):
    """
    Function to generate samples from signal contaminated by crosstalk.

    Parameters
    ----------
    ``signals_amount``: Amount of signals to be generated (default is 100).
    ``n_samples``: Amount of samples per signal to be generated (default is 4).
    ``sampling_period``: Time distance between samples (default is 25ns).

    Returns
    -------
    Dataframe containing signal shaping parameters and samples of approximated function according to given parameters.

    """
    
    Etrue_amp = np.abs(np.random.normal(loc=55000, scale=10000, size=signals_amount))
    Xt_amp = np.abs(np.random.normal(loc=4000, scale=1000, size=signals_amount))
    capacitance_clean_rand = np.random.normal(loc=50.0, scale=1.75, size=signals_amount)
    capacitance_xt_rand = np.random.normal(loc=47.0, scale=1.75, size=signals_amount)
    random_xt_delay = np.abs(np.random.normal(loc=10, scale=10, size=signals_amount))
    random_sampling_delay = np.abs(np.random.normal(loc=4, scale=4, size=signals_amount))
    data = np.stack([Etrue_amp, Xt_amp, capacitance_clean_rand, capacitance_xt_rand, random_xt_delay, random_sampling_delay], axis=1)
    columns = ["Etrue_amp", "Xt_amp", "capacitance_clean", "capacitance_xt", "delay_xt", "delay_sampling"]
    signals_df = pd.DataFrame(data, columns=columns)
    samples_columns = [f"S_{i}" for i in range(1, n_samples+1)]
    for i in range(n_samples):
        clean_samples = np.array([signals_df["Etrue_amp"][j]*cellFunction(sampling_period*(i+1) + signals_df["delay_sampling"][j],
                                                                          signals_df["capacitance_clean"][j]) for j in range(signals_amount)])
        xt_samples = np.array([signals_df["Xt_amp"][j]*XTalk(sampling_period*(i+1) + signals_df["delay_xt"][j] + signals_df["delay_sampling"][j],
                                                             signals_df["capacitance_xt"][j]) for j in range(signals_amount)])
        samples = clean_samples + xt_samples
        signals_df[samples_columns[i]] = samples
    
    return signals_df
    